In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import pandas as pd
import geopandas as gpd
import mytools

In [7]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20200616.csv',encoding='gbk'))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
gongcan_use = gongcan.loc[gongcan['RRU经度']>100]

In [9]:
gongcan_use_p = mytools.gisn.add_points(gongcan_use,'RRU经度','RRU纬度')

In [64]:
data = pd.read_clipboard()

In [65]:
data.head(1)

,规划站号,解决建议,置换入站点需求来源,经度,纬度
0,xuhao1,置换到G_五峰傅家堰集中安置点-HFH（110.456337 30.295415）,分公司上报弱覆盖,110.456337,30.295415


In [66]:
data_use = mytools.gisn.points_coverage_merge(data,'经度','纬度')

In [67]:
# data_use.to_clipboard()

In [68]:
data_use['制式'] = 'FDD900'

In [69]:
#修改倍数

In [70]:
data_use_juli = mytools.gisn.changjing_distance(data_use,'区域类型','制式',beishu=1.8)

In [71]:
# data_use_juli['distance']=650

In [72]:
data_use_juli.head(1)

,规划站号,解决建议,置换入站点需求来源,经度,纬度,区域类型,制式,distance
0,xuhao1,置换到G_五峰傅家堰集中安置点-HFH（110.456337 30.295415）,分公司上报弱覆盖,110.456337,30.295415,农村,FDD900,3960.0


In [73]:
data_buffer = mytools.gisn.add_buffer_df(data_use_juli,'经度','纬度','distance')

In [74]:
data_sjoin = gpd.sjoin(data_buffer,gongcan_use_p)

In [75]:
data_sjoin.groupby('规划站号').count().shape

(6, 70)

In [76]:
data_res = data_sjoin[['规划站号', '小区CGI(*)','区域类型']]

In [77]:
ruancai_cel = pd.read_excel('G:/1-规划/MRO/1、原始数据/摸底测试相关/2020年/5月/汇总双套数据5月.xlsx',encoding='gbk')

In [78]:
ruancai_cel.columns

Index(['CGI', '城市', 'Cell Name', '总采样点数', '低于-110采样点数', 'MR覆盖率'], dtype='object')

In [79]:
ruancai_cel['fgl'] = 1- ruancai_cel['低于-110采样点数']/ruancai_cel['总采样点数']

In [80]:
ruancai_cel.loc[ruancai_cel['fgl']<0.9,'弱覆盖小区']=1

In [81]:
data_res_v2 = data_res.merge(ruancai_cel,how='left',left_on='小区CGI(*)',right_on='CGI')

In [82]:
liuliang = pd.read_csv('G:/1-规划/流量/20200518LIULIANG.CSV',encoding='gbk')

In [83]:
liuliang_use = liuliang[[ 'CGI', '空口上行业务字节数','空口下行业务字节数']]

In [84]:
liuliang_use['流量总'] = liuliang_use['空口上行业务字节数'] +liuliang_use['空口下行业务字节数'] 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
liuliang_use_2 =  liuliang_use[[ 'CGI', '流量总']]

In [86]:
data_res_v3 = data_res_v2.merge(liuliang_use_2,how='left',left_on='小区CGI(*)',right_on='CGI')

In [87]:
data_res_v3.head(1)

,规划站号,小区CGI(*),区域类型,CGI_x,城市,Cell Name,总采样点数,低于-110采样点数,MR覆盖率,fgl,弱覆盖小区,CGI_y,流量总
0,xuhao1,460-00-121458-65,农村,460-00-121458-65,宜昌,五峰火山-HFH-2,0.0,0.0,NaN,NaN,NaN,460-00-121458-65,4742.308173


In [88]:
data_res_v3.loc[data_res_v3['总采样点数'].notna(),'有采样点小区数']=1

In [89]:
data_res_v4 = data_res_v3.groupby('规划站号').agg(
    {'总采样点数':'sum','低于-110采样点数':'sum','小区CGI(*)':'count','有采样点小区数':'sum',
     '弱覆盖小区':'count','流量总':'sum'}).reset_index()

In [90]:
data_res_v5 = data_res_v4.merge(data_use[['规划站号','区域类型']],how='left',on='规划站号')

In [93]:
data_res_v5.to_clipboard()

In [92]:
data_res_v3.to_clipboard()

In [103]:
data_res[data_res['规划站号']=='xuhao2']

,规划站号,小区CGI(*),区域类型


In [105]:
data_buffer

,规划站号,解决建议,置换入站点需求来源,经度,纬度,区域类型,制式,distance,geometry
0,xuhao1,置换到G_五峰傅家堰集中安置点-HFH（110.456337 30.295415）,分公司上报弱覆盖,110.456337,30.295415,农村,FDD900,3960.0,"POLYGON ((110.49725 30.29746, 110.49728 30.293..."
1,xuhao2,置换到G_枝江三宁乙二醇空分区-HFH（111.609662 30.393528）,省网优投诉,111.609662,30.393528,县城,FDD900,720.0,"POLYGON ((111.61712 30.39384, 111.61712 30.393..."
2,xuhao3,置换到G_枝江三宁乙二醇综合楼-HFH（111.607482 30.400893）,省网优投诉,111.607482,30.400893,县城,FDD900,720.0,"POLYGON ((111.61494 30.40120, 111.61494 30.400..."
3,xuhao4,置换到G_夷陵新妇幼-HFH(111.338348 30.756324),分公司上报弱覆盖,111.338348,30.756324,主城区,FDD900,540.0,"POLYGON ((111.34396 30.75657, 111.34396 30.756..."
4,xuhao5,置换到S_夷陵新妇幼-HFH(111.338348 30.756324),分公司上报弱覆盖,111.338348,30.756324,主城区,FDD900,540.0,"POLYGON ((111.34396 30.75657, 111.34396 30.756..."
5,xuhao6,置换到S_开发区环球港-HFH（111.324007 30.679548）,分公司上报弱覆盖,111.324007,30.679548,主城区,FDD900,540.0,"POLYGON ((111.32962 30.67979, 111.32962 30.679..."
6,xuhao7,置换到S_开发区三峡果蔬,省网优投诉,111.338540,30.697280,一般城区,FDD900,774.0,"POLYGON ((111.34658 30.69763, 111.34658 30.696..."
